In [1]:
import pandas as pd
import numpy as np
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Embedding
import re

In [2]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\shari\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\shari\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\shari\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
train = pd.read_csv('train.csv')
train

,Tweets,label
0,Asked #ChatGPT about what it thinks are the pr...,Positive
1,#ChatGPT tornado has already traveled around t...,Neutral
2,This is a great explanation of why #EVs are mo...,Positive
3,‘if you need to write a box-ticking social med...,Positive
4,Just saw an AI tool making my coffee for me. \...,Positive
...,...,...
765,NaN,NaN
766,NaN,NaN
767,NaN,NaN
768,NaN,NaN


# Dataset preprocessing

In [4]:
train.isnull().sum()

Tweets    171
label     171
dtype: int64

In [5]:
train = train.dropna()
train.isnull().sum()

Tweets    0
label     0
dtype: int64

In [6]:
train.duplicated().sum()
train = train.drop_duplicates()

In [7]:
len(train['Tweets'])

599

In [8]:
for i in range(0,599):
    train['Tweets'][i]=train['Tweets'][i].lower()

In [9]:
train

,Tweets,label
0,asked #chatgpt about what it thinks are the pr...,Positive
1,#chatgpt tornado has already traveled around t...,Neutral
2,this is a great explanation of why #evs are mo...,Positive
3,‘if you need to write a box-ticking social med...,Positive
4,just saw an ai tool making my coffee for me. \...,Positive
...,...,...
594,4. gpt’s ai capabilities are also more accessi...,Positive
595,3. keyboardx also provides a more secure envir...,Positive
596,2. keyboardx allows users to quickly and easil...,Positive
597,"i asked chatgpt why the app i’m working on, ke...",Positive


In [10]:
train['Tweets'][0]

'asked #chatgpt about what it thinks are the problems associated with urbanisation in india &amp; the solutions to be adopted to boost public transport use in our cities. interesting set of responses. \n\n#econtwitter #urbaneconomics @sselvan https://t.co/ytyd2ypxdh'

In [11]:
train['Tweets'][0]=re.sub(r"http\S+|www\S+|https\S+", '',train['Tweets'][0] , flags=re.MULTILINE)
train['Tweets'][0]

'asked #chatgpt about what it thinks are the problems associated with urbanisation in india &amp; the solutions to be adopted to boost public transport use in our cities. interesting set of responses. \n\n#econtwitter #urbaneconomics @sselvan '

In [12]:
train['Tweets'][150]

'"turnitin is the go-to #software to catch students #cheating. now it\'s focused on a potential cat-and-mouse game with openai\'s new #chatgpt #chatbot."\n#twlz #ki @hav_hendrik\n\n…usinessinsider-com.cdn.ampproject.org/c/s/www.busine…'

In [13]:
for i in range(599):
    train['Tweets'][i]=re.sub(r"http\S+|www\S+|https\S+", '',train['Tweets'][i] , flags=re.MULTILINE)

In [14]:
train['Tweets'][150]

'"turnitin is the go-to #software to catch students #cheating. now it\'s focused on a potential cat-and-mouse game with openai\'s new #chatgpt #chatbot."\n#twlz #ki @hav_hendrik\n\n…usinessinsider-com.cdn.ampproject.org/c/s/'

In [15]:
for i in range(599):
    train['Tweets'][i] = re.sub(r'@[\w]*', '', train['Tweets'][i])
    train['Tweets'][i] = re.sub(r'\W', ' ', train['Tweets'][i])
    train['Tweets'][i] = re.sub(r'\d', ' ', train['Tweets'][i])

In [16]:
train['Tweets'][150]

' turnitin is the go to  software to catch students  cheating  now it s focused on a potential cat and mouse game with openai s new  chatgpt  chatbot    twlz  ki    usinessinsider com cdn ampproject org c s '

In [17]:
for i in range(599):
    train['Tweets'][i] = word_tokenize(train['Tweets'][i])

In [18]:
train['Tweets'][150]

['turnitin',
 'is',
 'the',
 'go',
 'to',
 'software',
 'to',
 'catch',
 'students',
 'cheating',
 'now',
 'it',
 's',
 'focused',
 'on',
 'a',
 'potential',
 'cat',
 'and',
 'mouse',
 'game',
 'with',
 'openai',
 's',
 'new',
 'chatgpt',
 'chatbot',
 'twlz',
 'ki',
 'usinessinsider',
 'com',
 'cdn',
 'ampproject',
 'org',
 'c',
 's']

In [19]:
train['new']=None
train['preprocessed_tweets']=None
train

,Tweets,label,new,preprocessed_tweets
0,"[asked, chatgpt, about, what, it, thinks, are,...",Positive,None,None
1,"[chatgpt, tornado, has, already, traveled, aro...",Neutral,None,None
2,"[this, is, a, great, explanation, of, why, evs...",Positive,None,None
3,"[if, you, need, to, write, a, box, ticking, so...",Positive,None,None
4,"[just, saw, an, ai, tool, making, my, coffee, ...",Positive,None,None
...,...,...,...,...
594,"[gpt, s, ai, capabilities, are, also, more, ac...",Positive,None,None
595,"[keyboardx, also, provides, a, more, secure, e...",Positive,None,None
596,"[keyboardx, allows, users, to, quickly, and, e...",Positive,None,None
597,"[i, asked, chatgpt, why, the, app, i, m, worki...",Positive,None,None


In [20]:
stop_words = set(stopwords.words('english'))

In [21]:
lemmatizer = WordNetLemmatizer()

In [22]:
tokens=[]
for i in range(599):
    for token in train['Tweets'][i]:
        if token not in stop_words:
            tokens.append(token)
    train['new'][i]=tokens
    tokens=[]

In [23]:
train

,Tweets,label,new,preprocessed_tweets
0,"[asked, chatgpt, about, what, it, thinks, are,...",Positive,"[asked, chatgpt, thinks, problems, associated,...",None
1,"[chatgpt, tornado, has, already, traveled, aro...",Neutral,"[chatgpt, tornado, already, traveled, around, ...",None
2,"[this, is, a, great, explanation, of, why, evs...",Positive,"[great, explanation, evs, efficient, energy, s...",None
3,"[if, you, need, to, write, a, box, ticking, so...",Positive,"[need, write, box, ticking, social, media, pos...",None
4,"[just, saw, an, ai, tool, making, my, coffee, ...",Positive,"[saw, ai, tool, making, coffee, chatgpt, ai, a...",None
...,...,...,...,...
594,"[gpt, s, ai, capabilities, are, also, more, ac...",Positive,"[gpt, ai, capabilities, also, accessible, keyb...",None
595,"[keyboardx, also, provides, a, more, secure, e...",Positive,"[keyboardx, also, provides, secure, environmen...",None
596,"[keyboardx, allows, users, to, quickly, and, e...",Positive,"[keyboardx, allows, users, quickly, easily, ac...",None
597,"[i, asked, chatgpt, why, the, app, i, m, worki...",Positive,"[asked, chatgpt, app, working, keyboardx, bett...",None


In [24]:
for i in range(599):
    tokens=[]
    for token in train['new'][i]:
        token = lemmatizer.lemmatize(token,'v')
        tokens.append(token)
    train['new'][i]=tokens

In [25]:
train

,Tweets,label,new,preprocessed_tweets
0,"[asked, chatgpt, about, what, it, thinks, are,...",Positive,"[ask, chatgpt, think, problems, associate, urb...",None
1,"[chatgpt, tornado, has, already, traveled, aro...",Neutral,"[chatgpt, tornado, already, travel, around, wo...",None
2,"[this, is, a, great, explanation, of, why, evs...",Positive,"[great, explanation, evs, efficient, energy, s...",None
3,"[if, you, need, to, write, a, box, ticking, so...",Positive,"[need, write, box, tick, social, media, post, ...",None
4,"[just, saw, an, ai, tool, making, my, coffee, ...",Positive,"[saw, ai, tool, make, coffee, chatgpt, ai, aim...",None
...,...,...,...,...
594,"[gpt, s, ai, capabilities, are, also, more, ac...",Positive,"[gpt, ai, capabilities, also, accessible, keyb...",None
595,"[keyboardx, also, provides, a, more, secure, e...",Positive,"[keyboardx, also, provide, secure, environment...",None
596,"[keyboardx, allows, users, to, quickly, and, e...",Positive,"[keyboardx, allow, users, quickly, easily, acc...",None
597,"[i, asked, chatgpt, why, the, app, i, m, worki...",Positive,"[ask, chatgpt, app, work, keyboardx, better, u...",None


In [26]:
for i in range(599):
    tokens = train['new'][i]
    preprocessed_tweet = ' '.join(tokens)
    train['preprocessed_tweets'][i] = preprocessed_tweet

In [27]:
train

,Tweets,label,new,preprocessed_tweets
0,"[asked, chatgpt, about, what, it, thinks, are,...",Positive,"[ask, chatgpt, think, problems, associate, urb...",ask chatgpt think problems associate urbanisat...
1,"[chatgpt, tornado, has, already, traveled, aro...",Neutral,"[chatgpt, tornado, already, travel, around, wo...",chatgpt tornado already travel around world se...
2,"[this, is, a, great, explanation, of, why, evs...",Positive,"[great, explanation, evs, efficient, energy, s...",great explanation evs efficient energy source ...
3,"[if, you, need, to, write, a, box, ticking, so...",Positive,"[need, write, box, tick, social, media, post, ...",need write box tick social media post use esta...
4,"[just, saw, an, ai, tool, making, my, coffee, ...",Positive,"[saw, ai, tool, make, coffee, chatgpt, ai, aim...",saw ai tool make coffee chatgpt ai aimagic
...,...,...,...,...
594,"[gpt, s, ai, capabilities, are, also, more, ac...",Positive,"[gpt, ai, capabilities, also, accessible, keyb...",gpt ai capabilities also accessible keyboardx ...
595,"[keyboardx, also, provides, a, more, secure, e...",Positive,"[keyboardx, also, provide, secure, environment...",keyboardx also provide secure environment gpt ...
596,"[keyboardx, allows, users, to, quickly, and, e...",Positive,"[keyboardx, allow, users, quickly, easily, acc...",keyboardx allow users quickly easily access gp...
597,"[i, asked, chatgpt, why, the, app, i, m, worki...",Positive,"[ask, chatgpt, app, work, keyboardx, better, u...",ask chatgpt app work keyboardx better use gpt ...


# Vectorising

In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [29]:
tfidf = TfidfVectorizer(max_features=5000,ngram_range=(1,3))

In [30]:
preprocessed_tweets = []
for i in range(599):
    preprocessed_tweet = ' '.join(train['new'][i])
    preprocessed_tweets.append(preprocessed_tweet)

In [31]:
x = tfidf.fit_transform(preprocessed_tweets).toarray()
x

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [32]:
x.shape

(599, 5000)

In [33]:
from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()

In [34]:
train['label'] = encoder.fit_transform(train['label'])

In [35]:
train

,Tweets,label,new,preprocessed_tweets
0,"[asked, chatgpt, about, what, it, thinks, are,...",2,"[ask, chatgpt, think, problems, associate, urb...",ask chatgpt think problems associate urbanisat...
1,"[chatgpt, tornado, has, already, traveled, aro...",1,"[chatgpt, tornado, already, travel, around, wo...",chatgpt tornado already travel around world se...
2,"[this, is, a, great, explanation, of, why, evs...",2,"[great, explanation, evs, efficient, energy, s...",great explanation evs efficient energy source ...
3,"[if, you, need, to, write, a, box, ticking, so...",2,"[need, write, box, tick, social, media, post, ...",need write box tick social media post use esta...
4,"[just, saw, an, ai, tool, making, my, coffee, ...",2,"[saw, ai, tool, make, coffee, chatgpt, ai, aim...",saw ai tool make coffee chatgpt ai aimagic
...,...,...,...,...
594,"[gpt, s, ai, capabilities, are, also, more, ac...",2,"[gpt, ai, capabilities, also, accessible, keyb...",gpt ai capabilities also accessible keyboardx ...
595,"[keyboardx, also, provides, a, more, secure, e...",2,"[keyboardx, also, provide, secure, environment...",keyboardx also provide secure environment gpt ...
596,"[keyboardx, allows, users, to, quickly, and, e...",2,"[keyboardx, allow, users, quickly, easily, acc...",keyboardx allow users quickly easily access gp...
597,"[i, asked, chatgpt, why, the, app, i, m, worki...",2,"[ask, chatgpt, app, work, keyboardx, better, u...",ask chatgpt app work keyboardx better use gpt ...


In [36]:
y=train['label'].values
y

array([2, 1, 2, 2, 2, 0, 2, 2, 2, 0, 1, 2, 0, 0, 0, 1, 0, 2, 1, 2, 2, 1,
       2, 2, 2, 2, 2, 2, 2, 2, 1, 0, 1, 1, 2, 0, 2, 2, 2, 2, 2, 2, 2, 1,
       1, 0, 2, 0, 2, 1, 0, 1, 0, 1, 0, 2, 0, 2, 2, 1, 1, 0, 2, 2, 0, 1,
       0, 2, 0, 1, 2, 2, 0, 1, 0, 0, 2, 2, 1, 2, 2, 0, 2, 2, 2, 2, 2, 0,
       2, 2, 0, 0, 2, 0, 2, 0, 2, 1, 2, 0, 2, 1, 0, 0, 0, 2, 2, 0, 1, 0,
       2, 2, 2, 0, 0, 0, 1, 2, 0, 1, 0, 2, 0, 0, 2, 0, 2, 1, 1, 2, 0, 2,
       1, 1, 2, 0, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 0, 0, 2, 2, 2, 2, 2,
       0, 2, 0, 0, 2, 0, 2, 2, 0, 0, 1, 2, 1, 2, 2, 0, 1, 2, 2, 2, 1, 0,
       2, 2, 0, 2, 2, 0, 0, 2, 2, 1, 2, 2, 0, 2, 0, 2, 0, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 0, 2, 0, 2, 2, 1, 0, 2, 0, 2, 2, 0, 0, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 0, 1, 0, 2, 2, 0, 0, 2, 2, 2, 0, 0,
       2, 2, 2, 2, 1, 0, 2, 2, 0, 2, 0, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 1,
       0, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1,
       2, 2, 1, 2, 2, 2, 2, 0, 2, 2, 0, 0, 2, 0, 2,

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(preprocessed_tweets)
vocab_size = len(tokenizer.word_index) + 1 
print("Vocabulary size:", vocab_size)

# lstm model

In [37]:
model = Sequential([
    Embedding(input_dim=3014, output_dim=50, input_length=5000),
    LSTM(units=100),
    Dense(3, activation='softmax')  # Assuming 3 classes (0, 1, 2)
])

In [38]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 5000, 50)          150700    
                                                                 
 lstm (LSTM)                 (None, 100)               60400     
                                                                 
 dense (Dense)               (None, 3)                 303       
                                                                 
Total params: 211403 (825.79 KB)
Trainable params: 211403 (825.79 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [39]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [40]:
model.fit(X_train, y_train, epochs=1, batch_size=64)

8/8 [==============================] - 439s 55s/step - loss: 1.0538 - accuracy: 0.5344
